In [16]:
!pip install torch-geometric==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909879 sha256=11b81e82f882e539e4bfb11c3d1633104d886d9100360ed3b1e47822c6377e23
  Stored in directory: /root/.cache/pip/wheels/5f/66/21/18ca6ca537cd87a2f01e1c7504922a4bb5a9c2ff9dd36cbe1e
Successfully built torch-geometric
  Attempting uninstall: torch-geometric
    Found existing installation: torch_geometric 2.5.2
    Uninstalling torch_geometric-2.5.2:
      Successfully uninstalled torch_geometric-2.5.2


In [12]:
!nvidia-smi

Sun Aug  4 08:20:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0              31W /  70W |    105MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINConv
from torch_geometric.utils import scatter
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset

from tqdm.notebook import tqdm

In [17]:
dataset = TUDataset(root='/tmp/nci1', name='NCI1')
dataset = dataset.shuffle()
n_train = len(dataset) // 10 * 9
n_test = len(dataset) - n_train
train_dataset = dataset[:n_train]
test_dataset = dataset[n_train:]

In [18]:
# グラフ同型ネットワークの定義
class MLP(nn.Module):
    def __init__(self, in_d, mid_d, out_d):
        super(MLP, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(in_d, mid_d),
            nn.BatchNorm1d(mid_d),
            nn.ReLU(),
            nn.Linear(mid_d, out_d),
            nn.BatchNorm1d(out_d),
            nn.ReLU()
        )

    def forward(self, input):
        return self.main(input)


class GIN(torch.nn.Module):
    def __init__(self, in_d, mid_d, out_d):
        super(GIN, self).__init__()

        self.first_layer = MLP(in_d, mid_d, mid_d)
        self.conv1 = GINConv(MLP(mid_d, mid_d, mid_d))
        self.conv2 = GINConv(MLP(mid_d, mid_d, mid_d))
        self.conv3 = GINConv(MLP(mid_d, mid_d, mid_d))
        self.predictor = nn.Sequential(
            nn.Linear(mid_d, mid_d),
            nn.BatchNorm1d(mid_d),
            nn.ReLU(),
            nn.Linear(mid_d, out_d),
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.first_layer(x)
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        graph_embeddings = scatter(x, batch, dim=0, reduce='max')
        res = self.predictor(graph_embeddings)
        return F.log_softmax(res, dim=1)

In [26]:
model = GIN(dataset.num_node_features, 32, dataset.num_classes)

In [27]:
# model.cuda()
model = model.to(device)

In [28]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-4)

In [29]:
def train(epoch):
    model.train()
    loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    for i in tqdm(range(epoch), total=epoch):
        for data in loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()

In [30]:
%time
train(500)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


  0%|          | 0/500 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "


KeyboardInterrupt: 

In [25]:
model.eval()
acc = 0
for data in DataLoader(test_dataset, batch_size=32):
    data = data.to(device)
    pred = model(data).max(1)[1]
    acc += pred.eq(data.y).sum().item()
acc / len(test_dataset)

0.7883211678832117